In [1]:
import os, collections

from Bio import pairwise2
from Bio.Seq import Seq

import numpy as np
import pandas as pd

from data import AllBooks, Book
from utils import most_frequent

from align_config import dss_version, sp_version, QSP_SCROLLS
from align_functions import make_alignments, collect_matching_words

from tf.app import use

MT = use('etcbc/bhsa')
Fmt, Tmt, Lmt = MT.api.F, MT.api.T, MT.api.L

# SP should be specific version, because for that version there are predictions of the matres
SP = use('dt-ucph/sp:clone', checkout='clone', version=sp_version, provenanceSpec=dict(moduleSpecs=[]))
Fsp, Tsp, Lsp = SP.api.F, SP.api.T, SP.api.L

DSS = use('etcbc/dss', version=dss_version)
Fdss, Tdss, Ldss = DSS.api.F, DSS.api.T, DSS.api.L

C:\Users\geitb\anaconda3\envs\hebrew_spelling\Lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
C:\Users\geitb\anaconda3\envs\hebrew_spelling\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.20,100
chapter,187,2135.78,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399391,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
# file names:
# 'ptca_qal.csv'
# 'ptcp_qal.csv'
# 'infc_qal_triliteral.csv'
# 'infc_qal_lamed_he.csv'
# 'infa_qal.csv'
# 'niph_hiph_pe_yod.csv'
# 'hiphil_triliteral_with_hireq.csv'

MATRES_FILE = 'niph_hiph_pe_yod.csv'
MATRES_FILE

'niph_hiph_pe_yod.csv'

In [3]:
PENTATEUCH_BOOKS = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy']
ALL_BOOK_NAMES = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]

# Prepare MT and SP texts

Produce dictionary mt_sp_matches which has mt words nodes as keys and matching word numbers from SP as values.

In [4]:
# prepare mt and sp books
MANUSCRIPTS = ['MT', 'SP']
all_books = AllBooks()
for book_name in ALL_BOOK_NAMES:
    book = Book('MT', book_name, Fmt, Tmt, Lmt)
    all_books.data[('MT', book_name)] = book
    
    if book_name in PENTATEUCH_BOOKS:
        book = Book('SP', book_name, Fsp, Tsp, Lsp)
        all_books.data[('SP', book_name)] = book

# Match words

In [5]:
dat = pd.read_csv(os.path.join('../data', MATRES_FILE), sep='\t')

# remove non-hireq cases in case of triliteral hiphil
if MATRES_FILE == 'hiphil_triliteral_with_hireq.csv':
    dat = dat[dat.has_hireq == '1']

# dat_dss means non-MT, SP is included
dat_dss = dat[~dat.scroll.isin(['MT'])]
scroll_book_combinations = list(set(zip(dat_dss.scroll, dat_dss.book)))
dat.shape

(2332, 36)

In [6]:
for manuscript, book_name in scroll_book_combinations:
    if manuscript == 'MT':
        continue
    elif manuscript == 'SP':
        book = Book(manuscript, book_name, Fsp, Tsp, Lsp)
        all_books.data[(manuscript, book_name)] = book
    else:
        book = Book(manuscript, book_name, Fdss, Tdss, Ldss)
        all_books.data[(manuscript, book_name)] = book

In [7]:
def make_matching_word_dict(book_name, all_books):

    all_match_dicts = {}

    matching_book = book_name
    matching_manuscripts = [scr for scr, bo in all_books.data.keys() if bo == matching_book]
    #print(matching_manuscripts)

    for man1 in matching_manuscripts:
        for man2 in matching_manuscripts:
            if man1 != man2:
            
                man1_verse_texts = all_books.data[(man1, matching_book)].verse_text_dict
                man2_verse_texts = all_books.data[(man2, matching_book)].verse_text_dict

                alignments_dict = make_alignments(man1_verse_texts, man2_verse_texts)
    
                man1_word2char = all_books.data[(man1, matching_book)].word2char
                man2_word2char = all_books.data[(man2, matching_book)].word2char
 
                matching_words_dict = collect_matching_words(alignments_dict, man1_word2char, man2_word2char)

                matches = {}
                for man1_word, man2_list in matching_words_dict.items():
                    man2_word = most_frequent(man2_list)
                    matches[man1_word] = man2_word
                all_match_dicts[((man1, matching_book), (man2, matching_book))] = matches
                
    return all_match_dicts, matching_manuscripts

In [8]:
def read_dataset(file):

    return pd.read_csv(file, sep='\t')

In [9]:
class MaterData:
    def __init__(self, man1, man2, section, lex, mater_val1, mater_val2, tf_id1, tf_id2, g_cons1, g_cons2):
        self.man1 = man1
        self.man2 = man2
        self.section = section
        self.lex = lex
        self.mater_val1 = mater_val1
        self.mater_val2 = mater_val2
        self.tf_id1 = tf_id1
        self.tf_id2 = tf_id2
        self.g_cons1 = g_cons1
        self.g_cons2 = g_cons2

In [10]:
def collect_matching_cases(matching_manuscripts, matching_book, dat):

    all_mater_datasets = {}

    for man in matching_manuscripts:
        mater_data = dat[(dat.book == matching_book) & (dat.scroll == man)]
        all_mater_datasets[(man, matching_book)] = mater_data
        
    return all_mater_datasets

In [11]:
def collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, matching_book):

    manuscript_mater_match = collections.defaultdict(list)
    manuscripts = set()

    for idx, man in enumerate(matching_manuscripts):
        for idx2, man2 in enumerate(matching_manuscripts):
            if idx < idx2:
            
                matching_ids = all_match_dicts[((man, matching_book), (man2, matching_book))]
                man_data = all_mater_datasets[(man, matching_book)]
                man2_data = all_mater_datasets[(man2, matching_book)]
                for _, row in man_data.iterrows():
                
                    tf_id = row.tf_id
                    lex, typ, has_vl = row.lex, row.type, row.has_vowel_letter
                    g_cons1 = row.g_cons
                    section = (row.book, row.chapter, row.verse)
                    if man == 'SP':
                        tf_id = tf_id - 100000

                    matching_tf_id = matching_ids.get(tf_id, None)
                
                    if not matching_tf_id:
                        continue
                    
                    if man2 == 'SP':
                        matching_tf_id = matching_tf_id + 100000
                
                    man2_row = man2_data[(man2_data.tf_id == matching_tf_id) & (man2_data.lex == lex) & (man2_data.type == typ)]
                    
                    if not man2_row.shape[0]:
                        continue
                    has_vl2 = man2_row.has_vowel_letter.iloc[0]
                    g_cons2 = man2_row.g_cons.iloc[0]
                 
                    mater_data = MaterData(man, man2, section, lex, has_vl, has_vl2, tf_id, matching_tf_id, g_cons1, g_cons2)
                
                    manuscript_mater_match[man].append(mater_data)
                    manuscripts.add(man)
                    manuscripts.add(man2)
                    
    return manuscript_mater_match, manuscripts

In [12]:
def register_similarities_with_mt(manuscripts, mt_ids, manuscript_mater_match):

    mater_value_dict = {0: -1,
                    1: 1}

    mater_match_array = np.zeros((len(manuscripts), len(mt_ids)))

    for dat_object in manuscript_mater_match['MT']:
        
        other_man = dat_object.man2
        mt_tfid = dat_object.tf_id1
        other_man_tfid = dat_object.tf_id2
    
        mt_mater = dat_object.mater_val1
        other_man_mater = dat_object.mater_val2
    
        mt_idx = man2idx['MT']
        other_man_idx = man2idx[other_man]
    
        mt_mater_value = mater_value_dict[mt_mater]
        other_man_mater_value = mater_value_dict[other_man_mater]
    
        mt_tf_id = mt_tf2idx[mt_tfid]
    
        mater_match_array[mt_idx, mt_tf_id] = mt_mater_value
        mater_match_array[other_man_idx, mt_tf_id] = other_man_mater_value
        
    return mater_match_array

In [13]:
def get_parallels(manuscript_mater_match, hif_match_dict):
    for scroll in manuscript_mater_match.keys():
        for dat_object in manuscript_mater_match[scroll]:
            man1 = dat_object.man1
            man2 = dat_object.man2
            if man1 == 'MT' or man2 == 'MT':
            
                hif_match_dict[dat_object.tf_id2] = dat_object.tf_id1
                hif_match_dict[dat_object.tf_id1] = dat_object.tf_id2
        
    return hif_match_dict


In [14]:
def count_parallel_cases(mater_match_array):

    mater_arr = np.zeros((2, mater_match_array.shape[1]))

    for col_idx in range(mater_match_array.shape[1]):
        col = mater_match_array[:, col_idx]
        col_counts = collections.Counter(col)
        with_vowel_count = col_counts.get(1, 0)
        without_vowel_count = col_counts.get(-1, 0)
        mater_arr[0, col_idx] = with_vowel_count
        mater_arr[1, col_idx] = without_vowel_count
        
    return mater_arr

In [15]:
hif_match_dict = {}

for book in ALL_BOOK_NAMES:
    print(book)
    all_match_dicts, matching_manuscripts = make_matching_word_dict(book, all_books)
    all_mater_datasets = collect_matching_cases(matching_manuscripts, book, dat)
    manuscript_mater_match, manuscripts = collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, book)
    
    hif_match_dict = get_parallels(manuscript_mater_match, hif_match_dict)
    print(len(hif_match_dict))
    

Genesis
14
Exodus
88
Leviticus
102
Numbers
106
Deuteronomy
157
Joshua
163
Judges
163
1_Samuel
171
2_Samuel
185
1_Kings
187
2_Kings
187
Isaiah
507
Jeremiah
519
Ezekiel
525
Hosea
531
Joel
540
Amos
546
Obadiah
550
Jonah
558
Micah
568
Nahum
570
Habakkuk
574
Zephaniah
586
Haggai
586
Zechariah
588
Malachi
594
Psalms
708
Job
708
Proverbs
710
Ruth
714
Song_of_songs
714
Ecclesiastes
714
Lamentations
718
Esther
718
Daniel
718
Ezra
718
Nehemiah
718
1_Chronicles
718
2_Chronicles
718


In [16]:
all_scrolls = set(dat_dss.scroll) 
non_qsp_scrolls = all_scrolls.difference(QSP_SCROLLS)
non_qsp_data = dat_dss[dat_dss.scroll.isin(non_qsp_scrolls)]
non_qsp_data = non_qsp_data[non_qsp_data.scroll != 'SP']

qsp_data = dat_dss[~dat_dss.scroll.isin(non_qsp_scrolls)]

non_qsp_pent = non_qsp_data[non_qsp_data.book.isin(PENTATEUCH_BOOKS)]
non_qsp_pent.shape

(84, 36)

In [17]:
full_count = collections.defaultdict(int)
scrolls = set()
scroll_counts = collections.defaultdict(int)

sp_mt_dict = collections.defaultdict(lambda: collections.defaultdict(int))

for index, row in qsp_data.iterrows():
    tf_id = (row.tf_id),
    scroll = row.scroll, 
    has_vl = row.has_vowel_letter
    book, ch, ve = row.book, row.chapter, row.verse
    g_cons = row.g_cons
    lex = row.lex
    prefix = row.prefix
    reconstr = row.rec_signs
    vs = row.vs
    vt = row.vt
    
    
    #if tf_id[0] in hif_match_dict and has_vl != corr_row.has_vowel_letter.iloc[0]:
    if tf_id[0] not in hif_match_dict:
        continue
    corr_id = hif_match_dict[tf_id[0]]
    corr_row = dat[dat.tf_id == corr_id]
        #if has_vl == corr_row.has_vowel_letter.iloc[0] and has_vl == 0:
        
    print(tf_id[0], scroll, corr_row.scroll.iloc[0], row.chapter, row.verse, vs, vt, row.lex, g_cons, corr_row.g_cons.iloc[0], 
                  book, prefix, corr_row.prefix.iloc[0], has_vl, corr_row.has_vowel_letter.iloc[0], row.rec_signs)
        
    mt_full = corr_row.has_vowel_letter.iloc[0]
    scroll_full = has_vl
    full_count[(scroll_full, mt_full)] += 1
    scrolls.add(scroll)
    scroll_counts[scroll] += 1
            
    #else:
    #    #print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
    #    if scroll[0] != 'SP' and lex[0] == 'J':
    #           print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
    #    continue
    

print(full_count)
print(len((scrolls)))
print(scroll_counts)

1891380 ('1Q4',) MT 9 28 hif impf JY>[ JWY>NW HWY>TNW Deuteronomy nan nan 1 1 nnnnnr
1894937 ('1Qisaa',) MT 1 5 hif impf JSP[ TWSJPW TWSJPW Isaiah nan nan 1 1 nnnnnn
1894993 ('1Qisaa',) MT 1 8 nif ptca JTR[ NTRT NWTRH Isaiah W W 0 1 nnnn
1895012 ('1Qisaa',) MT 1 9 hif perf JTR[ HWTJR HWTJR Isaiah nan nan 1 1 nnnnn
1895073 ('1Qisaa',) MT 1 13 hif impf JSP[ TWSJPW TWSJPW Isaiah nan nan 1 1 nnnnnn
1895139 ('1Qisaa',) MT 1 17 hif infa JVB[ HJVJB HJVB Isaiah nan nan 1 1 nnnnn
1895408 ('1Qisaa',) MT 2 3 hif impf JRH[ JRWNW JRNW Isaiah W W 0 0 nnnnn
1895434 ('1Qisaa',) MT 2 4 hif perf JKX[ HWKJX HWKJX Isaiah W W 1 1 nnnnn
1896191 ('1Qisaa',) MT 4 3 nif ptca JTR[ NWTR NWTR Isaiah WH WH 1 1 nnnn
1896332 ('1Qisaa',) MT 5 5 hif impf JD<[ >WDJ< >WDJ<H Isaiah nan nan 1 1 nnnnn
1897527 ('1Qisaa',) MT 7 22 nif ptca JTR[ NWTR NWTR Isaiah H H 1 1 nnnn
1897679 ('1Qisaa',) MT 8 5 hif wayq JSP[ JWSP JSP Isaiah W W 1 0 nnnn
1897881 ('1Qisaa',) MT 8 15 nif perf JQC[ NWQCW NWQCW Isaiah W W 1 1 nnnnn
1898305 